In [ ]:
# demo conversion
import torch
import pickle
import numpy as np
from hparams import hparams
from utils import pad_seq_to_2
from utils import quantize_f0_numpy
from model import Generator_3 as Generator
from model import Generator_6 as F0_Converter


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
G = Generator(hparams).eval().to(device)
g_checkpoint = torch.load('assets/660000-G.ckpt', map_location=lambda storage, loc: storage)
G.load_state_dict(g_checkpoint['model'])

P = F0_Converter(hparams).eval().to(device)
p_checkpoint = torch.load('assets/640000-P.ckpt', map_location=lambda storage, loc: storage)
P.load_state_dict(p_checkpoint['model'])


metadata = pickle.load(open('assets/demo.pkl', "rb"))


sbmt_i = metadata[0]
emb_org = torch.from_numpy(sbmt_i[1]).to(device)
x_org, f0_org, len_org, uid_org = sbmt_i[2]        
uttr_org_pad, len_org_pad = pad_seq_to_2(x_org[np.newaxis,:,:], 192)
uttr_org_pad = torch.from_numpy(uttr_org_pad).to(device)
f0_org_pad = np.pad(f0_org, (0, 192-len_org), 'constant', constant_values=(0, 0))
f0_org_quantized = quantize_f0_numpy(f0_org_pad)[0]
f0_org_onehot = f0_org_quantized[np.newaxis, :, :]
f0_org_onehot = torch.from_numpy(f0_org_onehot).to(device)
uttr_f0_org = torch.cat((uttr_org_pad, f0_org_onehot), dim=-1)

sbmt_j = metadata[1]
emb_trg = torch.from_numpy(sbmt_j[1]).to(device)
x_trg, f0_trg, len_trg, uid_trg = sbmt_j[2]        
uttr_trg_pad, len_trg_pad = pad_seq_to_2(x_trg[np.newaxis,:,:], 192)
uttr_trg_pad = torch.from_numpy(uttr_trg_pad).to(device)
f0_trg_pad = np.pad(f0_trg, (0, 192-len_trg), 'constant', constant_values=(0, 0))
f0_trg_quantized = quantize_f0_numpy(f0_trg_pad)[0]
f0_trg_onehot = f0_trg_quantized[np.newaxis, :, :]
f0_trg_onehot = torch.from_numpy(f0_trg_onehot).to(device)

#리듬, 피치벡터만 봔환
with torch.no_grad():
    type_return = 1
    C_org, R_org, P_org = G(uttr_f0_org, uttr_org_pad, emb_org, type_return)
    
    #FlowModel에 주기 R_native, P_native = FlowModel(R_org, P_org)
    P_native = torch.load("assets/sample/pitch.pt")
    R_native = torch.load("assets/sample/rhythm.pt")
    R_native = R_native.to(device)
    P_native = P_native.to(device)
    
    spect_vc = []
    #mel_spectogram 생성
    type_return = 0
    x_1 = uttr_org_pad.transpose(2,1)
    encoder_outputs = torch.cat((C_org, R_native, P_native, 
                                     emb_org.unsqueeze(1).expand(-1,x_1.size(-1),-1)), dim=-1)
    x_identic_val = G(encoder_outputs,0, 0, type_return)
    uttr_trg = x_identic_val[0, : :].cpu().numpy()
    spect_vc.append( ('{}_{}'.format(sbmt_i[0]), uttr_trg ) )

In [ ]:
# spectrogram to waveform
import torch
import soundfile
import pickle
import os
from synthesis import build_model
from synthesis import wavegen

if not os.path.exists('results'):
    os.makedirs('results')

model = build_model().to(device)
checkpoint = torch.load("assets/checkpoint_step001000000_ema.pth", map_location=torch.device(device))
model.load_state_dict(checkpoint["state_dict"])

for spect in spect_vc:
    name = spect[0]
    c = spect[1]
    print(name)
    waveform = wavegen(model, c=c)   
    soundfile.write('results/'+name+'.wav', waveform, samplerate=16000)